In [1]:
from langchain_ollama import OllamaLLM

In [ ]:
llm = OllamaLLM(model="gemma3n:e4b")
llm.invoke("The first man on the moon was ...")

'The first man on the moon was **Neil Armstrong**. \n\nHe took his first steps on the lunar surface on July 20, 1969, as part of the Apollo 11 mission.\n'

In [14]:
for chunk in llm.stream("The first man on the moon was ..."):
    print(chunk, end="", flush=True)

The first man on the moon was **Neil Armstrong**. 

He made the historic first step onto the lunar surface on July 20, 1969, during the Apollo 11 mission.





In [21]:
from langchain_ollama import ChatOllama

chat_model = ChatOllama(model="gemma3n:e4b")

chat_model.invoke("Who was the first man on the moon?")

AIMessage(content='The first man on the moon was **Neil Armstrong**. \n\nHe took his first step onto the lunar surface on **July 20, 1969**, as part of the Apollo 11 mission. \n', additional_kwargs={}, response_metadata={'model': 'gemma3n:e4b', 'created_at': '2025-08-05T03:48:38.95474042Z', 'done': True, 'done_reason': 'stop', 'total_duration': 1092804392, 'load_duration': 74466676, 'prompt_eval_count': 18, 'prompt_eval_duration': 42738312, 'eval_count': 48, 'eval_duration': 974955230, 'model_name': 'gemma3n:e4b'}, id='run--8bea4237-2c36-4ede-933b-becc2f5093d3-0', usage_metadata={'input_tokens': 18, 'output_tokens': 48, 'total_tokens': 66})

In [22]:
for chunk in chat_model.stream("The first man on the moon was ..."):
    print(chunk.content, end="", flush=True)

The first man on the moon was **Neil Armstrong**. 

He made the historic first step onto the lunar surface on July 20, 1969, during the Apollo 11 mission.


In [57]:
from typing import Optional, Union
from pydantic import BaseModel, Field
from typing_extensions import Annotated, TypedDict

class Joke(TypedDict):
    """Joke to tell user."""
    setup: str = Field(description="The setup of the joke")
    punchline: str = Field(description="The punchline to the joke")
    rating: Optional[int] = Field(
        default=None, description="How funny the joke is, from 1 to 10"
    )


class ConversationalResponse(TypedDict):
    """Respond in a conversational manner. Be kind and helpful."""
    response: str = Field(description="A conversational response to the user's query and short")


class FinalResponse(TypedDict):
    final_output: Union[Joke, ConversationalResponse]

In [58]:
structured_chat_model = chat_model.with_structured_output(Joke)
structured_chat_model.invoke("Why don't scientists trust atoms?")

{'setup': "This is a humorous question playing on the word 'atoms' and 'atoms of trust'. It's a pun, not a serious scientific concern.",
 'punchline': 'Because they make up everything!',
 'rating': 5}

In [59]:
structured_chat_model = chat_model.with_structured_output(FinalResponse)
structured_chat_model.invoke("Why don't scientists trust atoms?")

{'final_output': {'response': 'This is a classic science joke! The punchline is: Because they make up everything!'}}

In [60]:
structured_chat_model = chat_model.with_structured_output(FinalResponse)
structured_chat_model.invoke("what is love?")

{'final_output': {'response': "That's a question humans have pondered for millennia! There's no single, easy answer, as love is a complex and multifaceted emotion. Here's a breakdown of different perspectives:\n\n**From a Psychological Perspective:**\n*   **Attachment Theory:**  Love is deeply rooted in our early childhood experiences with caregivers.  Secure attachment leads to trusting, healthy relationships.  Different attachment styles (secure, anxious, avoidant) influence how we express and experience love.\n*   **Neurochemistry:**  Love triggers the release of chemicals like dopamine (pleasure, reward), oxytocin (bonding, trust), and vasopressin (long-term bonding). These chemicals create feelings of euphoria, connection, and commitment.\n*   **Different Types of Love:** Psychologists often distinguish between different types of love:\n    *   **Romantic Love:**  Characterized by passion, intimacy, and commitment.  Often involves intense emotions and a desire for closeness.\n    

In [65]:
# Few shot prompt
from langchain_core.prompts import ChatPromptTemplate

system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") and the final punchline (the response to "<setup> who?").

Here are some examples of jokes:

example_user: Tell me a joke about planes
example_assistant: {{"setup": "Why don't planes ever get tired?", "punchline": "Because they have rest wings!", "rating": 2}}

example_user: Tell me another joke about planes
example_assistant: {{"setup": "Cargo", "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!", "rating": 10}}

example_user: Now about caterpillars
example_assistant: {{"setup": "Caterpillar", "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!", "rating": 5}}"""

prompt = ChatPromptTemplate.from_messages([("system", system), ("human", "{input}")])

few_shot_structured_llm = prompt | structured_chat_model
few_shot_structured_llm.invoke("what's something funny about Ladyboy?")

{'final_output': {'setup': 'Ladyboy',
  'punchline': "Ladyboy, you're so fabulous you make unicorns jealous!",
  'rating': 8}}

In [69]:
from langchain_core.messages import AIMessage, HumanMessage, ToolMessage

examples = [
    HumanMessage("Tell me a joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Why don't planes ever get tired?",
                    "punchline": "Because they have rest wings!",
                    "rating": 2,
                },
                "id": "1",
            }
        ],
    ),
    # Most tool-calling models expect a ToolMessage(s) to follow an AIMessage with tool calls.
    ToolMessage("", tool_call_id="1"),
    # Some models also expect an AIMessage to follow any ToolMessages,
    # so you may need to add an AIMessage here.
    HumanMessage("Tell me another joke about planes", name="example_user"),
    AIMessage(
        "",
        name="example_assistant",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Cargo",
                    "punchline": "Cargo 'vroom vroom', but planes go 'zoom zoom'!",
                    "rating": 10,
                },
                "id": "2",
            }
        ],
    ),
    ToolMessage("", tool_call_id="2"),
    HumanMessage("Now about caterpillars", name="example_user"),
    AIMessage(
        "",
        tool_calls=[
            {
                "name": "joke",
                "args": {
                    "setup": "Caterpillar",
                    "punchline": "Caterpillar really slow, but watch me turn into a butterfly and steal the show!",
                    "rating": 5,
                },
                "id": "3",
            }
        ],
    ),
    ToolMessage("", tool_call_id="3"),
]
system = """You are a hilarious comedian. Your specialty is knock-knock jokes. \
Return a joke which has the setup (the response to "Who's there?") \
and the final punchline (the response to "<setup> who?")."""

prompt = ChatPromptTemplate.from_messages(
    [("system", system), ("placeholder", "{examples}"), ("human", "{input}")]
)
few_shot_structured_llm = prompt | structured_chat_model
few_shot_structured_llm.invoke({"input": "see ya later alligator", "examples": examples})

{'final_output': {'setup': 'Caterpillar who?',
  'punchline': 'See ya later, *later*!',
  'rating': 4}}